# Setup and Execution Guide RAGAS

This notebook evaluates RAG performance with the RAGAS framework.

1. **API Key**

   * Create a `.env` file in the project root
   * Add: `OPENAI_API_KEY=your_key_here`

2. **Input Data**

   * Provide a JSON file with generated answers from RAGnaroX
   * Must include: `user_input`, `response`, `retrieved_contexts`, `reference` 
   * Typically stored in `results/` 

3. **Configuration (Cell 3)**

   * `dataset_folder`: path to results
   * `output_json`: JSON filename
   * `n`: number of entries to evaluate

4. **Output**

   * Console: metrics (Context Precision/Recall, Faithfulness, Answer Relevancy)
   * File: results appended to `results.md`

In [ ]:
import os, json
from ragas.llms import LangchainLLMWrapper
from ragas import evaluate, EvaluationDataset
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="./.env")

# Print the loaded API key for debugging
openai_api_key = os.environ.get('OPENAI_API_KEY')
print(f"OPENAI_API_KEY loaded: {openai_api_key}")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables") # Set your key here or via environment

In [ ]:
# Set your dataset folder here (e.g. 'single-hop-RAG-dataset', 'multi-hop-RAG-dataset', 'lang-RAG-dataset')
dataset_folder = 'results'  # Change as needed
output_json = 'single_qwen30B_multi5_bge.json'  # Change as needed
output_json_path = os.path.join(dataset_folder, output_json)
results_md_path = os.path.join(dataset_folder, 'results.md')

llm = ChatOpenAI(model='gpt-4o-mini')

evaluator_llm = LangchainLLMWrapper(llm)
with open(output_json_path, 'r', encoding='utf-8') as f:
    evaluation_dataset = json.load(f)
n = 1000  # Change as needed
evaluation_dataset = evaluation_dataset[:n]
evaluation_dataset = EvaluationDataset.from_list(evaluation_dataset)
print(f'Evaluation dataset loaded with {len(evaluation_dataset)} entries.')
result = evaluate(dataset=evaluation_dataset, llm=evaluator_llm)
print(result)

In [ ]:
with open(results_md_path, 'a', encoding='utf-8') as f:
    f.write(f'## Results for {dataset_folder}\n')
    for metric, value in result.items():
        f.write(f'- {metric}: {value}\n')
    f.write('\n')
print(f'Results for {dataset_folder} saved to {results_md_path}')